<a href="https://colab.research.google.com/github/mihirukongahage/HomeAlone2020_Kaggle/blob/master/HomeAlone2020_V3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HomeAlone2020 

In [0]:
import pandas as pd
import io
import numpy as np

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Train.csv to Train (1).csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded['Train.csv']))

In [0]:
df

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Lakshadweep,LAKSHADWEEP,DR. MOHAMMED SADIQUE KP,0,JD(U),MALE,0,64,ST,Post Graduate,"Rs 4,97,83,111\n ~ 4 Crore+","Rs 28,87,460\n ~ 28 Lacs+",55189
1,Lakshadweep,LAKSHADWEEP,MOHAMMED FAIZAL PP,1,NCP,MALE,2,43,ST,Post Graduate,"Rs 9,38,641\n ~ 9 Lacs+",Rs 0\n ~,55189
2,Lakshadweep,LAKSHADWEEP,HAMDULLAH SAYEED,0,INC,MALE,0,36,ST,Graduate Professional,"Rs 9,50,47,817\n ~ 9 Crore+","Rs 4,45,771\n ~ 4 Lacs+",55189
3,Daman & Diu,DAMAN & DIU,LALUBHAI BABUBHAI PATEL,1,BJP,MALE,0,63,GENERAL,8th Pass,"Rs 21,39,93,750\n ~ 21 Crore+","Rs 99,69,514\n ~ 99 Lacs+",121740
4,Jammu & Kashmir,LADAKH,SAJJAD HUSSAIN,0,IND,MALE,0,37,ST,Post Graduate,"Rs 1,09,000\n ~ 1 Lacs+",Rs 0\n ~,179232
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Bihar,HAJIPUR (SC),ARVIND PASWAN,0,IND,MALE,0,33,SC,12th Pass,"Rs 12,05,263\n ~ 12 Lacs+","Rs 47,000\n ~ 47 Thou+",1823664
1496,Bihar,HAJIPUR (SC),PASHU PATI KUMAR PARAS,1,LJP,MALE,1,66,SC,Graduate Professional,"Rs 6,28,34,200\n ~ 6 Crore+","Rs 67,44,375\n ~ 67 Lacs+",1823664
1497,Bihar,HAJIPUR (SC),UMESH DAS,0,BSP,MALE,0,47,SC,10th Pass,"Rs 1,10,21,433\n ~ 1 Crore+","Rs 15,00,000\n ~ 15 Lacs+",1823664
1498,Bihar,HAJIPUR (SC),RAJGIR PASWAN,0,BJKVP,MALE,0,41,SC,Literate,"Rs 10,82,600\n ~ 10 Lacs+",Rs 0\n ~,1823664


In [0]:
# Check for null values
df.isnull().any()

STATE              False
CONSTITUENCY       False
NAME               False
WINNER             False
PARTY              False
GENDER             False
CRIMINAL\nCASES    False
AGE                False
CATEGORY           False
EDUCATION          False
ASSETS             False
LIABILITIES        False
TOTAL ELECTORS     False
dtype: bool

In [0]:
# Drop invalid rows
df = df[df['ASSETS'] != 'Not Available']
df = df[df['ASSETS'] != 'Nil']

df = df[df['LIABILITIES'] != 'Not Available']
df = df[df['LIABILITIES'] != 'NIL']

In [0]:
# Convert ASSETS and LIABILITIES columns to int64 

df['ASSETS'] = df['ASSETS'].str.split('\n').str[0]
df['ASSETS'] = df['ASSETS'].str.split('Rs ').str[1]
df['ASSETS'] = df['ASSETS'].str.replace(',', '')

df['LIABILITIES'] = df['LIABILITIES'].str.split('\n').str[0]
df['LIABILITIES'] = df['LIABILITIES'].str.split('Rs ').str[1]
df['LIABILITIES'] = df['LIABILITIES'].str.replace(',', '')

df['ASSETS'] = df['ASSETS'].astype(float).astype('int64')
df['LIABILITIES'] = df['LIABILITIES'].astype(float).astype('int64')

In [0]:
#Drop columns
df = df.drop(['STATE', 'CONSTITUENCY','NAME'], axis=1)

In [0]:
# Type conversion
df['CRIMINAL\nCASES'] = df['CRIMINAL\nCASES'].astype(float).astype('int64')

### Data Presentation

In [0]:
import matplotlib.pyplot as plt

In [0]:
# Heat map
f = plt.figure(figsize=(20, 16))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=12, rotation=90)
plt.yticks(range(df.shape[1]), df.columns, fontsize=12)
cb = plt.colorbar()

In [0]:
# Scatter plot
pd.plotting.scatter_matrix(df)

In [0]:
# individual plots
from matplotlib import pyplot
pyplot.scatter( df['WINNER'],df['PARTY'])

### One Hot Encoding

In [0]:
# One hot encode 
dfDummies1 = pd.get_dummies(df['GENDER'])
dfDummies2 = pd.get_dummies(df['PARTY'])
dfDummies3 = pd.get_dummies(df['CATEGORY'])
dfDummies4 = pd.get_dummies(df['EDUCATION'])

In [0]:
# Concatinate the dummies
df = pd.concat([df, dfDummies1,dfDummies2,dfDummies3,dfDummies4], axis=1)

In [0]:
# Drop unwanted columns
df = df.drop(['PARTY', 'GENDER','CATEGORY','EDUCATION'], axis=1)

### Imputing mean for missing

In [0]:
for_mean = df[df['LIABILITIES'] != 0]
lib_mean = for_mean["LIABILITIES"].mean()
df['LIABILITIES'] = df['LIABILITIES'].replace(0, int(lib_mean))

In [0]:
df

### Splitting and training the dataset for different classifiers 

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
# Split data
y = df['WINNER']
X = df.drop(['WINNER'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state = 21, stratify=y)

In [0]:
# Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=5)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
# Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
predicted= model.predict(X_test)
print (predicted)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train, y_train)
y_prend = knn.predict(X_test)
print(y_prend)
knn.score(X_test,y_test)

### Predicting for the test dataset 

In [0]:
uploaded2 = files.upload()

Saving Test.csv to Test (1).csv


In [0]:
test_df = pd.read_csv(io.BytesIO(uploaded2['Test.csv']))

In [0]:
test_df

,STATE,CONSTITUENCY,NAME,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Bihar,BHAGALPUR,AJAY KUMAR MANDAL,JD(U),MALE,0,41,GENERAL,8th Pass,"Rs 68,01,126\n ~ 68 Lacs+","Rs 53,64,747\n ~ 53 Lacs+",1823820
1,Bihar,BHAGALPUR,SHAILESH KUMAR ALIAS VULO MANDAL,RJD,MALE,0,43,GENERAL,12th Pass,"Rs 3,03,64,808\n ~ 3 Crore+","Rs 21,80,647\n ~ 21 Lacs+",1823820
2,Gujarat,BARDOLI,CHAUDHARI DR. TUSHARBHAI AMARSINHBHAI,INC,MALE,0,53,ST,Graduate Professional,"Rs 6,93,45,867\n ~ 6 Crore+",Rs 0\n ~,1826526
3,Gujarat,BARDOLI,PARBHUBHAI NAGARBHAI VASAVA,BJP,MALE,0,49,ST,Others,"Rs 2,83,50,217\n ~ 2 Crore+","Rs 36,76,869\n ~ 36 Lacs+",1826526
4,Uttarakhand,NAINITAL-UDHAMSINGH NAGAR,ER. NAVNEET AGARWAL,BSP,MALE,0,46,GENERAL,Graduate Professional,"Rs 5,00,00,000","Rs 3,53,52,263\n ~ 3 Crore+",1828433
...,...,...,...,...,...,...,...,...,...,...,...,...
492,Karnataka,BANGALORE NORTH,D.V. SADANANDA GOWDA,BJP,MALE,0,66,GENERAL,Graduate Professional,"Rs 20,93,84,539\n ~ 20 Crore+","Rs 10,43,02,653\n ~ 10 Crore+",2849250
493,Telangana,MALKAJGIRI,RAJASHEKAR REDDY MARRI,TRS,MALE,0,50,GENERAL,12th Pass,"Rs 56,79,40,114\n ~ 56 Crore+","Rs 9,14,08,506\n ~ 9 Crore+",3150313
494,Telangana,MALKAJGIRI,MAHENDER REDDY BONGUNOORI,JnP,MALE,0,43,GENERAL,10th Pass,"Rs 13,97,14,395\n ~ 13 Crore+","Rs 4,07,43,724\n ~ 4 Crore+",3150313
495,Telangana,MALKAJGIRI,ANUMULA REVANTH REDDY,INC,MALE,42,51,GENERAL,Graduate,"Rs 24,53,57,182\n ~ 24 Crore+","Rs 2,78,07,304\n ~ 2 Crore+",3150313


In [0]:
# Drop invalid rows
test_df = test_df[test_df['ASSETS'] != 'Not Available']
test_df = test_df[test_df['ASSETS'] != 'Nil']
test_df = test_df[test_df['ASSETS'] != '`']

test_df = test_df[test_df['LIABILITIES'] != 'Not Available']
test_df = test_df[test_df['LIABILITIES'] != 'NIL']

In [0]:
# Convert ASSETS and LIABILITIES columns to int64 

test_df['ASSETS'] = test_df['ASSETS'].str.split('\n').str[0]
test_df['ASSETS'] = test_df['ASSETS'].str.split('Rs ').str[1]
test_df['ASSETS'] = test_df['ASSETS'].str.replace(',', '')

test_df['LIABILITIES'] = test_df['LIABILITIES'].str.split('\n').str[0]
test_df['LIABILITIES'] = test_df['LIABILITIES'].str.split('Rs ').str[1]
test_df['LIABILITIES'] = test_df['LIABILITIES'].str.replace(',', '')

test_df['ASSETS'] = test_df['ASSETS'].astype(float).astype('int64')
test_df['LIABILITIES'] = test_df['LIABILITIES'].astype(float).astype('int64')

In [0]:
#Drop columns
test_df = test_df.drop(['STATE', 'CONSTITUENCY','NAME'], axis=1)

In [0]:
test_df['CRIMINAL\nCASES'] = test_df['CRIMINAL\nCASES'].astype(float).astype('int64')

In [0]:
# One hot encode 
test_dfDummies1 = pd.get_dummies(test_df['GENDER'])
test_dfDummies2 = pd.get_dummies(test_df['PARTY'])
test_dfDummies3 = pd.get_dummies(test_df['CATEGORY'])
test_dfDummies4 = pd.get_dummies(test_df['EDUCATION'])

In [0]:
# Concatinate the dummies
test_df = pd.concat([test_df, test_dfDummies1,test_dfDummies2,test_dfDummies3,test_dfDummies4], axis=1)

In [0]:
# Drop unwanted
test_df = test_df.drop(['PARTY', 'GENDER','CATEGORY','EDUCATION'], axis=1)

In [0]:
test_df['LIABILITIES'] = test_df['LIABILITIES'].replace(0, int(lib_mean))

In [0]:
# Bringing train and test dataset to same dimensions
test1= list(np.setdiff1d(list(test_df.columns) ,list(df.columns) ))
test2= list(np.setdiff1d(list(df.columns),list(test_df.columns)  ))
for col in test2:
    test_df[col] = 0
test_df = test_df.drop(test1, axis=1)

In [0]:
test_df = test_df.drop(['WINNER'], axis=1)

In [0]:
# Predicting the test dataset with KNN classifier
y_prend = knn.predict(test_df)

In [0]:
y_prend

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
np.savetxt("out2.csv", y_pred, delimiter=",")